In [1]:
import cv2 as cv
import numpy as np
from detect import detect
from PIL import Image
from utils import *

c:\Users\Ryann\Desktop\Topicos em Processamento Digital de Images\Projeto\car-detection\detect.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ch


Loaded checkpoint from epoch 3.



## **Single Shot Multibox Detector**

In [ ]:
train_test_split(os.getcwd() + '\\dataset')

In [ ]:
!python create_data_lists.py

### **Training**

In [ ]:
!python train.py

### **Evaluation**

In [ ]:
!python eval.py

### **Detection**

In [2]:
path_video = 'dataset\\videos\\video08.mp4'

In [3]:
video_name = path_video.split('\\')[-1].split('.')[0]

In [4]:
frame_count = 0

In [5]:
cap = cv.VideoCapture(path_video)

create_results_folder('ssd',video_name)

while(True):
    ret, frame = cap.read()

    frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    if not ret: 
        break

    frame = Image.fromarray(frame)
    
    frame = detect(frame, min_score=0.2, max_overlap=0.5, top_k=200)

    frame = cv.cvtColor(np.array(frame), cv.COLOR_RGB2BGR)

    cv.imwrite(f'results\\ssd\\{video_name}\\frame{frame_count}.png', frame)
    frame_count += 1

    cv.imshow('Frame', frame)

    if cv.waitKey(60) & 0xff == 27:
        break

cap.release()
cv.destroyAllWindows()

c:\Users\Ryann\Desktop\Topicos em Processamento Digital de Images\Projeto\car-detection\model.py:501: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:28.)
  image_boxes.append(class_decoded_locs[1 - suppress])
c:\Users\Ryann\Desktop\Topicos em Processamento Digital de Images\Projeto\car-detection\model.py:503: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:28.)
  image_scores.append(class_scores[1 - suppress])


## **MOG2**

In [11]:
path_video = 'dataset\\videos\\video08.mp4'

In [12]:
video_name = path_video.split('\\')[-1].split('.')[0]

In [13]:
frame_count = 0

In [14]:
mog2 = cv.createBackgroundSubtractorMOG2(varThreshold=100, detectShadows=False)

In [15]:
cap = cv.VideoCapture(path_video)

create_results_folder('mog2',video_name)

while(True):
    ret, frame = cap.read()

    if not ret: 
        break
    
    fgmask = mog2.apply(frame)

    fgmask = cv.erode(fgmask, np.ones((5,5), np.uint8))
    fgmask = cv.dilate(fgmask, np.ones((41,41), np.uint8))
    
    fgmask = cv.morphologyEx(fgmask, cv.MORPH_CLOSE, np.ones((45, 45), np.uint8))

    _, binary_mask = cv.threshold(fgmask, 1, 255, cv.THRESH_BINARY)

    contours, _ = cv.findContours(binary_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    contour_sort = sorted(contours, key=cv.contourArea, reverse=True)

    for i, contour in enumerate(contours):
        x, y, w, h = cv.boundingRect(contour)

        area = cv.contourArea(contour)

        detected_width_range = 50 < (x + w) - x < 1000
        detected_height_range = (100 < (y + h) - y < 700)
        if (area >= 6000 and area <= 150000) and detected_height_range and detected_height_range:
            cv.rectangle(frame, (x,y), (x + w, y + h), (0,0,255), 2)

            cv.putText(frame, f"Carro", (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
    
    cv.imwrite(f'results\\mog2\\{video_name}\\frame{frame_count}.png', frame)
    frame_count += 1

    cv.imshow('Frame', frame)

    if cv.waitKey(60) & 0xff == 27:
        break

cap.release()
cv.destroyAllWindows()